In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization

In [ ]:
data=pd.read_csv('../input/Kannada-MNIST/train.csv')


In [ ]:
X_data= data.values[:600, 1:]
y_label=data.values[:600, 0]
n_classes=10
y_label = to_categorical(y_label, n_classes)

X_data = X_data.reshape((X_data.shape[0], 28, 28, 1))
from sklearn.model_selection import train_test_split
X_train_pre_process, X_test_pre_process, y_train_pre_process, y_test_pre_process = train_test_split(X_data, y_label ,test_size=0.2, random_state=42)


In [ ]:
def data_augment(X_train,y_train, batch_size):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator # for data augmentation

    datagen = ImageDataGenerator(zoom_range=0.1,
                                 width_shift_range=0.3,
                                 height_shift_range=0.3,
                                 rotation_range=5)
    datagen.fit(X_data)
    train_generator=datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
    i=0
    for img_batch in train_generator:
        x,y=img_batch
        X_train=np.concatenate((X_train,x), axis=0)
        y_train=np.concatenate((y_train,y), axis=0)
        i=i+1    
        if i >= batch_size:
            break
    return X_train, y_train

In [ ]:
X_train_gray, y_train_gray=data_augment(X_train_pre_process, y_train_pre_process,20)


In [ ]:
print(X_train_gray.shape)
print(y_train_gray.shape)

In [ ]:
def gray_rgb(X_data):
    X=[]

    for i in range(X_data.shape[0]):
        X_temp=X_data[i,:].reshape(28,28,1)
        X_temp_resize= tf.image.resize(X_temp, [94,94])
        X.append(np.concatenate((X_temp_resize,X_temp_resize,X_temp_resize),axis=-1))
    return X

In [ ]:
X_train=np.asarray(gray_rgb(X_train_gray))
X_test=np.asarray(gray_rgb(X_test_pre_process))

y_train=y_train_gray
y_test=y_test_pre_process

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
base_model=tf.keras.applications.MobileNetV2(
    input_shape=(94,94,3), alpha=1.0, include_top=False, weights='imagenet',
    input_tensor=None, pooling=None)

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
c=model.fit(x=X_train, y=y_train,epochs=30,batch_size=32, validation_data=(X_test,y_test), shuffle=True)

In [ ]:
loss = c.history['loss']
val_loss = c.history['val_loss']

len(loss)

train_epoch=range(1,31)
plt.plot(train_epoch,loss, label='loss')
plt.plot(train_epoch,val_loss, label='validation_loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
accuracy  = c.history['accuracy']
val_accuracy = c.history['val_accuracy']



train_epoch=range(1,31)
plt.plot(train_epoch,accuracy , label='accuracy')
plt.plot(train_epoch,val_accuracy , label='val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.legend()

text=('best accuracy' ,max(val_accuracy),)

